In [15]:
import pandas as pd
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import ast
import numpy as np

In [16]:
wiki_movies = pd.read_csv("wikipedia_movie_franchises.csv", index_col = 0)
imdb_movies = pd.read_csv("imdb_movies_db.csv", index_col = 0)

In [17]:
wiki_movies = wiki_movies.dropna(axis=0, how='any')
wiki_movies.reset_index(inplace = True)
wiki_movies["release_year"] = wiki_movies["release_year"].apply(lambda x : str(int(x)))

In [18]:
wiki_movies["movie_name"] = wiki_movies["movie_name"].apply(lambda x : x.strip())
wiki_movies["franchise_name"] = wiki_movies["franchise_name"].apply(lambda x : x.strip())

In [19]:
wiki_movies["movie_name"] = wiki_movies["movie_name"].apply(lambda x : x.replace("–", "-"))
wiki_movies["movie_name"] = wiki_movies["movie_name"].apply(lambda x : x.lower())

In [20]:
def lower_akas(row):
    if type(row) == str:
        row = row.lower()
        return row

In [21]:
imdb_movies["primaryTitle"] = imdb_movies["primaryTitle"].apply(lambda x : x.lower())
imdb_movies["originalTitle"] = imdb_movies["originalTitle"].apply(lambda x : x.lower())
imdb_movies["akas"] = imdb_movies["akas"].apply(lambda x : lower_akas(x))

In [22]:
wiki_filter = ["tconst", "titleType", "primaryTitle","originalTitle", "isAdult","startYear","endYear","runtimeMinutes","genres","averageRating","numVotes","akas","_merge", "all_names"]
imdb_filter = ["index", "franchise_name", "franchise_id", "movie_name", "release_year","_merge"]

In [23]:
imdb_copy = imdb_movies.copy()

In [29]:
imdb_copy

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,akas,all_names
8,tt0000009,movie,miss jerry,miss jerry,0,1894,\N,45,Romance,5.3,205.0,"[miss jerry, fräulein jerry]",[]
144,tt0000147,movie,the corbett-fitzsimmons fight,the corbett-fitzsimmons fight,0,1897,\N,100,"Documentary,News,Sport",5.3,469.0,"[the corbett-fitzsimmons fight, бой корбетта и...",[]
498,tt0000502,movie,bohemios,bohemios,0,1905,\N,100,\N,4.1,15.0,[bohemios],[]
570,tt0000574,movie,the story of the kelly gang,the story of the kelly gang,0,1906,\N,70,"Action,Adventure,Biography",6.0,823.0,"[the story of the kelly gang, kelly bandájának...",[]
587,tt0000591,movie,the prodigal son,l'enfant prodigue,0,1907,\N,90,Drama,4.4,20.0,"[l'enfant prodigue, the prodigal son]",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9724870,tt9916270,movie,il talento del calabrone,il talento del calabrone,0,2020,\N,84,Thriller,5.8,1414.0,"[il talento del calabrone, the talent of the h...",[]
9724915,tt9916362,movie,coven,akelarre,0,2020,\N,92,"Drama,History",6.4,5293.0,"[boszorkánygyülekezet, coven, les sorcières d'...",[]
9724947,tt9916428,movie,the secret of china,hong xing zhao yao zhong guo,0,2019,\N,\N,"Adventure,History,War",3.8,14.0,"[the secret of china, hong xing zhao yao zhong...",[]
9724999,tt9916538,movie,kuambil lagi hatiku,kuambil lagi hatiku,0,2019,\N,123,Drama,8.6,7.0,[kuambil lagi hatiku],[]


In [24]:
def filter_df(df, merge, cols):
    remainder = df[df["_merge"]== merge]
    remainder = remainder.drop(cols, axis = 1)
    return remainder

In [25]:
def extract_akas(row):
    # convert string in list format to list
    if type(row) == str:
        akas = ast.literal_eval(row)
        return akas

In [26]:
imdb_copy["akas"] = imdb_copy["akas"].apply(lambda x : extract_akas(x))

In [27]:
imdb_copy['all_names'] = np.empty((len(imdb_copy), 0)).tolist()

In [37]:
imdb_copy = imdb_copy.reset_index(drop=True)

In [38]:
for i in range(0, len(list(imdb_copy["primaryTitle"]))):
    imdb_copy['all_names'][i].append(imdb_copy['primaryTitle'][i])
    imdb_copy['all_names'][i].append(imdb_copy['originalTitle'][i])
    
    if type(imdb_copy['akas'][i]) == list:
        for x in imdb_copy['akas'][i]:
            imdb_copy['all_names'][i].append(x)
        
        
imdb_copy

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,akas,all_names
0,tt0000009,movie,miss jerry,miss jerry,0,1894,\N,45,Romance,5.3,205.0,"[miss jerry, fräulein jerry]","[miss jerry, miss jerry, miss jerry, fräulein ..."
1,tt0000147,movie,the corbett-fitzsimmons fight,the corbett-fitzsimmons fight,0,1897,\N,100,"Documentary,News,Sport",5.3,469.0,"[the corbett-fitzsimmons fight, бой корбетта и...","[the corbett-fitzsimmons fight, the corbett-fi..."
2,tt0000502,movie,bohemios,bohemios,0,1905,\N,100,\N,4.1,15.0,[bohemios],"[bohemios, bohemios, bohemios]"
3,tt0000574,movie,the story of the kelly gang,the story of the kelly gang,0,1906,\N,70,"Action,Adventure,Biography",6.0,823.0,"[the story of the kelly gang, kelly bandájának...","[the story of the kelly gang, the story of the..."
4,tt0000591,movie,the prodigal son,l'enfant prodigue,0,1907,\N,90,Drama,4.4,20.0,"[l'enfant prodigue, the prodigal son]","[the prodigal son, l'enfant prodigue, l'enfant..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
290234,tt9916270,movie,il talento del calabrone,il talento del calabrone,0,2020,\N,84,Thriller,5.8,1414.0,"[il talento del calabrone, the talent of the h...","[il talento del calabrone, il talento del cala..."
290235,tt9916362,movie,coven,akelarre,0,2020,\N,92,"Drama,History",6.4,5293.0,"[boszorkánygyülekezet, coven, les sorcières d'...","[coven, akelarre, boszorkánygyülekezet, coven,..."
290236,tt9916428,movie,the secret of china,hong xing zhao yao zhong guo,0,2019,\N,\N,"Adventure,History,War",3.8,14.0,"[the secret of china, hong xing zhao yao zhong...","[the secret of china, hong xing zhao yao zhong..."
290237,tt9916538,movie,kuambil lagi hatiku,kuambil lagi hatiku,0,2019,\N,123,Drama,8.6,7.0,[kuambil lagi hatiku],"[kuambil lagi hatiku, kuambil lagi hatiku, kua..."


In [39]:
imdb_copy = imdb_copy.explode("all_names")

In [40]:
all_merged = pd.merge(wiki_movies, imdb_copy, left_on=["movie_name", "release_year"], right_on=["all_names", "startYear"], how="outer", indicator = True)
wiki_remainder = filter_df(all_merged, "left_only", wiki_filter)
imdb_remainder = filter_df(all_merged, "right_only", imdb_filter)
all_merged[all_merged["_merge"]=="both"]

,index,franchise_id,franchise_name,movie_name,release_year,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,akas,all_names,_merge
12,12.0,f1,Coffin Joe,this night i'll possess your corpse,1967,tt0060380,movie,esta noite encarnarei no teu cadáver,esta noite encarnarei no teu cadáver,0.0,1967,\N,108,Horror,7.0,1952.0,"[esta noche poseeré tu cadáver, esta noite enc...",this night i'll possess your corpse,both
13,13.0,f1,Coffin Joe,the strange world of coffin joe,1968,tt0062943,movie,the strange world of coffin joe,o estranho mundo de zé do caixão,0.0,1968,\N,80,Horror,6.3,653.0,"[странный мир зе-из-гроба, the strange world o...",the strange world of coffin joe,both
14,13.0,f1,Coffin Joe,the strange world of coffin joe,1968,tt0062943,movie,the strange world of coffin joe,o estranho mundo de zé do caixão,0.0,1968,\N,80,Horror,6.3,653.0,"[странный мир зе-из-гроба, the strange world o...",the strange world of coffin joe,both
15,14.0,f1,Coffin Joe,awakening of the beast,1970,tt0062690,movie,awakening of the beast,o ritual dos sádicos,0.0,1970,\N,93,"Drama,Horror",5.8,1208.0,"[пробуждение зверя, ritual of the maniacs, o d...",awakening of the beast,both
16,14.0,f1,Coffin Joe,awakening of the beast,1970,tt0062690,movie,awakening of the beast,o ritual dos sádicos,0.0,1970,\N,93,"Drama,Horror",5.8,1208.0,"[пробуждение зверя, ritual of the maniacs, o d...",awakening of the beast,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16296,8226.0,f1342,Wong Fei Hung (Chinese films),huang fei hong zhi nan er dang bao guo,1993,tt0107161,movie,fist from shaolin,huang fei hong zhi nan er dang bao guo,0.0,1993,\N,94,"Action,Adventure,Comedy",6.5,95.0,"[wong fei hung ji naam yi dong bo gwok, huang ...",huang fei hong zhi nan er dang bao guo,both
16297,8227.0,f1342,Wong Fei Hung (Chinese films),huang fei hong dui huang fei hong,1993,tt0107160,movie,huang fei hong dui huang fei hong,huang fei hong dui huang fei hong,0.0,1993,\N,88,"Action,Comedy,Crime",6.0,144.0,"[wong fei hung dui wong fei hung, master wong ...",huang fei hong dui huang fei hong,both
16298,8227.0,f1342,Wong Fei Hung (Chinese films),huang fei hong dui huang fei hong,1993,tt0107160,movie,huang fei hong dui huang fei hong,huang fei hong dui huang fei hong,0.0,1993,\N,88,"Action,Comedy,Crime",6.0,144.0,"[wong fei hung dui wong fei hung, master wong ...",huang fei hong dui huang fei hong,both
16299,8227.0,f1342,Wong Fei Hung (Chinese films),huang fei hong dui huang fei hong,1993,tt0107160,movie,huang fei hong dui huang fei hong,huang fei hong dui huang fei hong,0.0,1993,\N,88,"Action,Comedy,Crime",6.0,144.0,"[wong fei hung dui wong fei hung, master wong ...",huang fei hong dui huang fei hong,both


In [41]:
wiki_remainder[wiki_remainder["franchise_id"] == "f1327"]

,index,franchise_id,franchise_name,movie_name,release_year
1328,7430.0,f1327,Marvel Cinematic Universe (various),deadpool 3,2024
14749,7425.0,f1327,Marvel Cinematic Universe (various),guardians of the galaxy vol. 3,2023
14750,7426.0,f1327,Marvel Cinematic Universe (various),the marvels,2023
14751,7427.0,f1327,Marvel Cinematic Universe (various),captain america: new world order,2024
14752,7428.0,f1327,Marvel Cinematic Universe (various),thunderbolts,2024
14753,7429.0,f1327,Marvel Cinematic Universe (various),blade,2024
14754,7431.0,f1327,Marvel Cinematic Universe (various),fantastic four,2025
14755,7432.0,f1327,Marvel Cinematic Universe (various),avengers: the kang dynasty,2025
14756,7433.0,f1327,Marvel Cinematic Universe (various),avengers: secret wars,2026


In [42]:
all_merged_both = all_merged[all_merged["_merge"]=="both"]
all_merged_both = all_merged_both.drop_duplicates(subset = ["index", "tconst"],keep = "first")
all_merged_both

,index,franchise_id,franchise_name,movie_name,release_year,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,akas,all_names,_merge
12,12.0,f1,Coffin Joe,this night i'll possess your corpse,1967,tt0060380,movie,esta noite encarnarei no teu cadáver,esta noite encarnarei no teu cadáver,0.0,1967,\N,108,Horror,7.0,1952.0,"[esta noche poseeré tu cadáver, esta noite enc...",this night i'll possess your corpse,both
13,13.0,f1,Coffin Joe,the strange world of coffin joe,1968,tt0062943,movie,the strange world of coffin joe,o estranho mundo de zé do caixão,0.0,1968,\N,80,Horror,6.3,653.0,"[странный мир зе-из-гроба, the strange world o...",the strange world of coffin joe,both
15,14.0,f1,Coffin Joe,awakening of the beast,1970,tt0062690,movie,awakening of the beast,o ritual dos sádicos,0.0,1970,\N,93,"Drama,Horror",5.8,1208.0,"[пробуждение зверя, ritual of the maniacs, o d...",awakening of the beast,both
18,16.0,f1,Coffin Joe,the bloody exorcism of coffin joe,1974,tt0071479,movie,exorcismo negro,exorcismo negro,0.0,1974,\N,100,Horror,6.0,274.0,"[exorcisme noir, black exorcism of coffin joe,...",the bloody exorcism of coffin joe,both
19,17.0,f1,Coffin Joe,the strange hostel of naked pleasures,1976,tt0074499,movie,the strange hostel of naked pleasures,a estranha hospedaria dos prazeres,0.0,1976,\N,81,Horror,4.7,289.0,"[a estranha hospedaria dos prazeres, the stran...",the strange hostel of naked pleasures,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16287,8221.0,f1342,Wong Fei Hung (Chinese films),martial club,1981,tt0083343,movie,martial club,wu guan,0.0,1981,\N,110,"Action,Drama",7.0,580.0,"[o clube das artes marciais, el club marcial d...",martial club,both
16291,8224.0,f1342,Wong Fei Hung (Chinese films),huang fei hong xiao zhuan,1992,tt0104459,movie,huang fei hong xiao zhuan,huang fei hong xiao zhuan,0.0,1992,\N,93,"Action,Comedy,Crime",5.9,184.0,"[wong fei hung siu chuen, huang fei hong xiao ...",huang fei hong xiao zhuan,both
16295,8226.0,f1342,Wong Fei Hung (Chinese films),huang fei hong zhi nan er dang bao guo,1993,tt0107161,movie,fist from shaolin,huang fei hong zhi nan er dang bao guo,0.0,1993,\N,94,"Action,Adventure,Comedy",6.5,95.0,"[wong fei hung ji naam yi dong bo gwok, huang ...",huang fei hong zhi nan er dang bao guo,both
16297,8227.0,f1342,Wong Fei Hung (Chinese films),huang fei hong dui huang fei hong,1993,tt0107160,movie,huang fei hong dui huang fei hong,huang fei hong dui huang fei hong,0.0,1993,\N,88,"Action,Comedy,Crime",6.0,144.0,"[wong fei hung dui wong fei hung, master wong ...",huang fei hong dui huang fei hong,both


In [43]:
no_year = pd.merge(wiki_remainder, imdb_remainder, left_on=["movie_name"], right_on=["all_names"], how="outer", indicator = True)
wiki_remainder = filter_df(all_merged, "left_only", wiki_filter)
imdb_remainder = filter_df(all_merged, "right_only", imdb_filter)
no_year

,index,franchise_id,franchise_name,movie_name,release_year,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,akas,all_names,_merge
0,0.0,f0,The Aldrich Family,what a life,1949,tt0032123,movie,what a life,what a life,0.0,1939,\N,75,"Comedy,Drama",6.9,93.0,"[what a life, a vida começa aos 14]",what a life,both
1,0.0,f0,The Aldrich Family,what a life,1949,tt0032123,movie,what a life,what a life,0.0,1939,\N,75,"Comedy,Drama",6.9,93.0,"[what a life, a vida começa aos 14]",what a life,both
2,0.0,f0,The Aldrich Family,what a life,1949,tt0032123,movie,what a life,what a life,0.0,1939,\N,75,"Comedy,Drama",6.9,93.0,"[what a life, a vida começa aos 14]",what a life,both
3,1.0,f0,The Aldrich Family,life with henry,1952,tt0033834,movie,life with henry,life with henry,0.0,1940,\N,80,"Comedy,Family,Music",6.0,49.0,"[life with henry, henry quería ir a alaska, he...",life with henry,both
4,1.0,f0,The Aldrich Family,life with henry,1952,tt0033834,movie,life with henry,life with henry,0.0,1940,\N,80,"Comedy,Family,Music",6.0,49.0,"[life with henry, henry quería ir a alaska, he...",life with henry,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1907206,NaN,NaN,NaN,NaN,NaN,tt9916538,movie,kuambil lagi hatiku,kuambil lagi hatiku,0.0,2019,\N,123,Drama,8.6,7.0,[kuambil lagi hatiku],kuambil lagi hatiku,right_only
1907207,NaN,NaN,NaN,NaN,NaN,tt9916730,movie,6 gunn,6 gunn,0.0,2017,\N,116,\N,8.3,10.0,"[६ गुण, 6 gunn]",6 gunn,right_only
1907208,NaN,NaN,NaN,NaN,NaN,tt9916730,movie,6 gunn,6 gunn,0.0,2017,\N,116,\N,8.3,10.0,"[६ गुण, 6 gunn]",6 gunn,right_only
1907209,NaN,NaN,NaN,NaN,NaN,tt9916730,movie,6 gunn,6 gunn,0.0,2017,\N,116,\N,8.3,10.0,"[६ गुण, 6 gunn]",6 gunn,right_only


In [44]:
no_year_both = no_year[no_year["_merge"]=="both"]
no_year_both = no_year_both.drop_duplicates(subset = ["index", "tconst"],keep = "first")
no_year_both

,index,franchise_id,franchise_name,movie_name,release_year,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,akas,all_names,_merge
0,0.0,f0,The Aldrich Family,what a life,1949,tt0032123,movie,what a life,what a life,0.0,1939,\N,75,"Comedy,Drama",6.9,93.0,"[what a life, a vida começa aos 14]",what a life,both
3,1.0,f0,The Aldrich Family,life with henry,1952,tt0033834,movie,life with henry,life with henry,0.0,1940,\N,80,"Comedy,Family,Music",6.0,49.0,"[life with henry, henry quería ir a alaska, he...",life with henry,both
6,2.0,f0,The Aldrich Family,henry aldrich for president,1952,tt0033708,movie,henry aldrich for president,henry aldrich for president,0.0,1941,\N,75,"Comedy,Family",6.6,146.0,"[henry aldrich para presidente, henry aldrich ...",henry aldrich for president,both
9,3.0,f0,The Aldrich Family,"henry aldrich, editor",1952,tt0034842,movie,"henry aldrich, editor","henry aldrich, editor",0.0,1942,\N,72,"Comedy,Drama,Family",6.4,150.0,"[henry periodista, henry aldrich, editor]","henry aldrich, editor",both
12,4.0,f0,The Aldrich Family,henry and dizzy,1952,tt0034844,movie,henry and dizzy,henry and dizzy,0.0,1942,\N,71,"Comedy,Family",7.2,58.0,[henry and dizzy],henry and dizzy,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4998,8068.0,f1340,Hopalong Cassidy (American-Western),fool's gold,1947,tt0140055,movie,fool's gold,fool's gold,0.0,1999,\N,95,Comedy,5.3,46.0,[fool's gold],fool's gold,both
5001,8068.0,f1340,Hopalong Cassidy (American-Western),fool's gold,1947,tt0415806,movie,fool's gold,fool's gold,0.0,2005,\N,90,Comedy,6.3,228.0,[fool's gold],fool's gold,both
5004,8068.0,f1340,Hopalong Cassidy (American-Western),fool's gold,1947,tt0770752,movie,fool's gold,fool's gold,0.0,2008,\N,112,"Action,Adventure,Comedy",5.7,81773.0,"[fool's gold, chasse au trésor, amor y tesoro,...",fool's gold,both
5007,8068.0,f1340,Hopalong Cassidy (American-Western),fool's gold,1947,tt0891302,movie,fool's gold,fool's gold,0.0,2003,\N,55,Documentary,5.8,24.0,[fool's gold],fool's gold,both


In [45]:
matched = pd.concat([all_merged_both, no_year_both])
matched = matched[matched["_merge"]=="both"]
matched.sort_values("numVotes", ascending = False).drop_duplicates(subset=["index", "franchise_id"], keep="first").sort_values(["index"])
matched

,index,franchise_id,franchise_name,movie_name,release_year,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,akas,all_names,_merge
12,12.0,f1,Coffin Joe,this night i'll possess your corpse,1967,tt0060380,movie,esta noite encarnarei no teu cadáver,esta noite encarnarei no teu cadáver,0.0,1967,\N,108,Horror,7.0,1952.0,"[esta noche poseeré tu cadáver, esta noite enc...",this night i'll possess your corpse,both
13,13.0,f1,Coffin Joe,the strange world of coffin joe,1968,tt0062943,movie,the strange world of coffin joe,o estranho mundo de zé do caixão,0.0,1968,\N,80,Horror,6.3,653.0,"[странный мир зе-из-гроба, the strange world o...",the strange world of coffin joe,both
15,14.0,f1,Coffin Joe,awakening of the beast,1970,tt0062690,movie,awakening of the beast,o ritual dos sádicos,0.0,1970,\N,93,"Drama,Horror",5.8,1208.0,"[пробуждение зверя, ritual of the maniacs, o d...",awakening of the beast,both
18,16.0,f1,Coffin Joe,the bloody exorcism of coffin joe,1974,tt0071479,movie,exorcismo negro,exorcismo negro,0.0,1974,\N,100,Horror,6.0,274.0,"[exorcisme noir, black exorcism of coffin joe,...",the bloody exorcism of coffin joe,both
19,17.0,f1,Coffin Joe,the strange hostel of naked pleasures,1976,tt0074499,movie,the strange hostel of naked pleasures,a estranha hospedaria dos prazeres,0.0,1976,\N,81,Horror,4.7,289.0,"[a estranha hospedaria dos prazeres, the stran...",the strange hostel of naked pleasures,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4998,8068.0,f1340,Hopalong Cassidy (American-Western),fool's gold,1947,tt0140055,movie,fool's gold,fool's gold,0.0,1999,\N,95,Comedy,5.3,46.0,[fool's gold],fool's gold,both
5001,8068.0,f1340,Hopalong Cassidy (American-Western),fool's gold,1947,tt0415806,movie,fool's gold,fool's gold,0.0,2005,\N,90,Comedy,6.3,228.0,[fool's gold],fool's gold,both
5004,8068.0,f1340,Hopalong Cassidy (American-Western),fool's gold,1947,tt0770752,movie,fool's gold,fool's gold,0.0,2008,\N,112,"Action,Adventure,Comedy",5.7,81773.0,"[fool's gold, chasse au trésor, amor y tesoro,...",fool's gold,both
5007,8068.0,f1340,Hopalong Cassidy (American-Western),fool's gold,1947,tt0891302,movie,fool's gold,fool's gold,0.0,2003,\N,55,Documentary,5.8,24.0,[fool's gold],fool's gold,both


In [46]:
matched = matched[["index", "franchise_id", "franchise_name", "movie_name", "tconst", "primaryTitle", "isAdult", "release_year", "runtimeMinutes", "genres", "averageRating", "numVotes"]]

In [47]:
matched.columns = matched.columns.str.replace('release_year', 'startYear')

In [48]:
matched["startYear"] = matched["startYear"].apply(lambda x : int(x))

C:\Users\35387\AppData\Local\Temp\ipykernel_67336\3960273086.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched["startYear"] = matched["startYear"].apply(lambda x : int(x))


In [49]:
imdb_final = pd.merge(matched, imdb_movies, on = "tconst", how = "outer")
imdb_final

,index,franchise_id,franchise_name,movie_name,tconst,primaryTitle_x,isAdult_x,startYear_x,runtimeMinutes_x,genres_x,...,primaryTitle_y,originalTitle,isAdult_y,startYear_y,endYear,runtimeMinutes_y,genres_y,averageRating_y,numVotes_y,akas
0,12.0,f1,Coffin Joe,this night i'll possess your corpse,tt0060380,esta noite encarnarei no teu cadáver,0.0,1967.0,108,Horror,...,esta noite encarnarei no teu cadáver,esta noite encarnarei no teu cadáver,0,1967,\N,108,Horror,7.0,1952.0,"['esta noche poseeré tu cadáver', 'esta noite ..."
1,5002.0,f822,Coffin' Joe,this night i will possess your corpse,tt0060380,esta noite encarnarei no teu cadáver,0.0,1967.0,108,Horror,...,esta noite encarnarei no teu cadáver,esta noite encarnarei no teu cadáver,0,1967,\N,108,Horror,7.0,1952.0,"['esta noche poseeré tu cadáver', 'esta noite ..."
2,13.0,f1,Coffin Joe,the strange world of coffin joe,tt0062943,the strange world of coffin joe,0.0,1968.0,80,Horror,...,the strange world of coffin joe,o estranho mundo de zé do caixão,0,1968,\N,80,Horror,6.3,653.0,"['странный мир зе-из-гроба', 'the strange worl..."
3,14.0,f1,Coffin Joe,awakening of the beast,tt0062690,awakening of the beast,0.0,1970.0,93,"Drama,Horror",...,awakening of the beast,o ritual dos sádicos,0,1970,\N,93,"Drama,Horror",5.8,1208.0,"['пробуждение зверя', 'ritual of the maniacs',..."
4,16.0,f1,Coffin Joe,the bloody exorcism of coffin joe,tt0071479,exorcismo negro,0.0,1974.0,100,Horror,...,exorcismo negro,exorcismo negro,0,1974,\N,100,Horror,6.0,274.0,"['exorcisme noir', 'black exorcism of coffin j..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290421,NaN,NaN,NaN,NaN,tt9916270,NaN,NaN,NaN,NaN,NaN,...,il talento del calabrone,il talento del calabrone,0,2020,\N,84,Thriller,5.8,1414.0,"['il talento del calabrone', 'the talent of th..."
290422,NaN,NaN,NaN,NaN,tt9916362,NaN,NaN,NaN,NaN,NaN,...,coven,akelarre,0,2020,\N,92,"Drama,History",6.4,5293.0,"['boszorkánygyülekezet', 'coven', ""les sorcièr..."
290423,NaN,NaN,NaN,NaN,tt9916428,NaN,NaN,NaN,NaN,NaN,...,the secret of china,hong xing zhao yao zhong guo,0,2019,\N,\N,"Adventure,History,War",3.8,14.0,"['the secret of china', 'hong xing zhao yao zh..."
290424,NaN,NaN,NaN,NaN,tt9916538,NaN,NaN,NaN,NaN,NaN,...,kuambil lagi hatiku,kuambil lagi hatiku,0,2019,\N,123,Drama,8.6,7.0,['kuambil lagi hatiku']


In [50]:
imdb_final = imdb_final[["index", "franchise_id", "franchise_name", "movie_name", "tconst", "primaryTitle_y", "isAdult_y", "startYear_y", "runtimeMinutes_y", "genres_y", "averageRating_y", "numVotes_y"]]
imdb_final

,index,franchise_id,franchise_name,movie_name,tconst,primaryTitle_y,isAdult_y,startYear_y,runtimeMinutes_y,genres_y,averageRating_y,numVotes_y
0,12.0,f1,Coffin Joe,this night i'll possess your corpse,tt0060380,esta noite encarnarei no teu cadáver,0,1967,108,Horror,7.0,1952.0
1,5002.0,f822,Coffin' Joe,this night i will possess your corpse,tt0060380,esta noite encarnarei no teu cadáver,0,1967,108,Horror,7.0,1952.0
2,13.0,f1,Coffin Joe,the strange world of coffin joe,tt0062943,the strange world of coffin joe,0,1968,80,Horror,6.3,653.0
3,14.0,f1,Coffin Joe,awakening of the beast,tt0062690,awakening of the beast,0,1970,93,"Drama,Horror",5.8,1208.0
4,16.0,f1,Coffin Joe,the bloody exorcism of coffin joe,tt0071479,exorcismo negro,0,1974,100,Horror,6.0,274.0
...,...,...,...,...,...,...,...,...,...,...,...,...
290421,NaN,NaN,NaN,NaN,tt9916270,il talento del calabrone,0,2020,84,Thriller,5.8,1414.0
290422,NaN,NaN,NaN,NaN,tt9916362,coven,0,2020,92,"Drama,History",6.4,5293.0
290423,NaN,NaN,NaN,NaN,tt9916428,the secret of china,0,2019,\N,"Adventure,History,War",3.8,14.0
290424,NaN,NaN,NaN,NaN,tt9916538,kuambil lagi hatiku,0,2019,123,Drama,8.6,7.0


In [51]:
imdb_final.columns = imdb_final.columns.str.replace('primaryTitle_y', 'primaryTitle')
imdb_final.columns = imdb_final.columns.str.replace('isAdult_y', 'isAdult')
imdb_final.columns = imdb_final.columns.str.replace('startYear_y', 'startYear')
imdb_final.columns = imdb_final.columns.str.replace('runtimeMinutes_y', 'runtimeMinutes')
imdb_final.columns = imdb_final.columns.str.replace('genres_y', 'genres')
imdb_final.columns = imdb_final.columns.str.replace('averageRating_y', 'averageRating')
imdb_final.columns = imdb_final.columns.str.replace('numVotes_y', 'numVotes')

In [52]:
imdb_final

,index,franchise_id,franchise_name,movie_name,tconst,primaryTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes
0,12.0,f1,Coffin Joe,this night i'll possess your corpse,tt0060380,esta noite encarnarei no teu cadáver,0,1967,108,Horror,7.0,1952.0
1,5002.0,f822,Coffin' Joe,this night i will possess your corpse,tt0060380,esta noite encarnarei no teu cadáver,0,1967,108,Horror,7.0,1952.0
2,13.0,f1,Coffin Joe,the strange world of coffin joe,tt0062943,the strange world of coffin joe,0,1968,80,Horror,6.3,653.0
3,14.0,f1,Coffin Joe,awakening of the beast,tt0062690,awakening of the beast,0,1970,93,"Drama,Horror",5.8,1208.0
4,16.0,f1,Coffin Joe,the bloody exorcism of coffin joe,tt0071479,exorcismo negro,0,1974,100,Horror,6.0,274.0
...,...,...,...,...,...,...,...,...,...,...,...,...
290421,NaN,NaN,NaN,NaN,tt9916270,il talento del calabrone,0,2020,84,Thriller,5.8,1414.0
290422,NaN,NaN,NaN,NaN,tt9916362,coven,0,2020,92,"Drama,History",6.4,5293.0
290423,NaN,NaN,NaN,NaN,tt9916428,the secret of china,0,2019,\N,"Adventure,History,War",3.8,14.0
290424,NaN,NaN,NaN,NaN,tt9916538,kuambil lagi hatiku,0,2019,123,Drama,8.6,7.0


In [53]:
imdb_final.to_csv("movies_with_franchises.csv")

# Unused code

In [ ]:
list1 = list(matched["index"])
found_mask = []
for i in range(0, len(list(wiki_movies["index"]))):
    if i in list1:
        found_mask.append(True)
    else:
        found_mask.append(False)
    
#print(list1)
print(wiki_movies[found_mask])

In [ ]:
# apply this mask function at each step of the matching process and see how many found rows there are

In [ ]:
3450 + 4749

In [ ]:
matched["index"].value_counts()

In [ ]:
both_test = matched[matched["_merge"]=="both"].sort_values(by = "index")

In [ ]:
dict(both_test["franchise_name"].value_counts())

In [ ]:
# look into moving all names into one list, then searching with date
# With remainder, search without date and see what happens

In [ ]:
# losing some values in both data frames for unknown reason when using merge.
# starts with 6405 rows in wiki and 290239 rows in imdb
# 3307 
#              wiki  |  imdb 
# start     |  6405  |  290239
# found     |  3088  |    3088
# remaining |  3307  |  287199
# total     |  6395  |  290287
# variance  |   -10  |     +48

### Checks for movies in dataframes

In [ ]:
test = both_test[both_test["franchise_name"].str.contains("Edgar Wallace Mysteries", na=False, flags=re.IGNORECASE, regex=True)]
test.head(50)

In [ ]:
test = wiki_movies[wiki_movies["franchise_name"].str.contains("Edgar Wallace Mysteries", na=False, flags=re.IGNORECASE, regex=True)]
test

In [ ]:
test = imdb_movies[imdb_movies["primaryTitle"].str.contains("star wars", na=False, flags=re.IGNORECASE, regex=True)]
test

In [ ]:
test = matched[matched["index"] == 5191.0]
test

In [ ]:
test = merged_df[merged_df["index"].str.contains("star wars", na=False, flags=re.IGNORECASE, regex=True)]
test


In [ ]:
imdb_movies['titles'] = imdb_movies.apply(', '.join(imdb_movies.columns["primaryTitle", "originalTitle", "akas"]), axis = 1)
imdb_movies

In [ ]:
def fuzzy_match(wiki, imdb, threshold):
    # iterate through each row in wiki
    # 
    # 
    # 
    # 
    # 
    # 
    # 

In [ ]:
fuzzy_match(wiki_movies, imdb_movies, 95)